# Example: 

In [4]:
using Flux
using Random

# Create a small neural network with random weights
begin
    Random.seed!(1234);

    model = Chain(
        Dense(2 => 10, relu),
        Dense(10 => 50, relu),
        Dense(50 => 20, relu),
        Dense(20 => 5, relu),
        Dense(5 => 1)
    )
end

# Create the workers
using Distributed
addprocs(4)
@everywhere using Gurobi

# In order to prevent Gurobi obtaining a new license for each solve
@everywhere ENV = Ref{Gurobi.Env}()

@everywhere function init_env()
    global ENV
    ENV[] = Gurobi.Env()
end

for worker in workers()
    fetch(@spawnat worker init_env())
end

# Regardless of the solver, this must be defined
@everywhere using JuMP

@everywhere function set_solver!(jump)
    set_optimizer(jump, () -> Gurobi.Optimizer(ENV[]))
    set_silent(jump)
end

# Set upper and lower input bounds
init_U = [1.0, 1.0];
init_L = [-1.0, -1.0];

@everywhere using Gogeta

# Create a JuMP model from the neural network with parallel bound tightening.
jump = Model()
@time U, L = NN_formulate!(jump, model, init_U, init_L; bound_tightening="standard", silent=false, parallel=true);

LoadError: MethodError: no method matching (::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}})()

[0mClosest candidates are:
[0m  (::Chain)([91m::Any[39m)
[0m[90m   @[39m [36mFlux[39m [90m~/.julia/packages/Flux/Wz6D4/src/layers/[39m[90m[4mbasic.jl:51[24m[39m
